In [ ]:
import os, math, random, glob, zipfile, pickle
import pandas as fp
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import scipy as sp
import PIL.Image as Image
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D, Dropout, GlobalAveragePooling2D, AveragePooling2D, BatchNormalization, ZeroPadding2D, Activation
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend as K
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, auc, confusion_matrix

In [ ]:
# Copy the dataset from Google Drive to Colab
# from google.colab import drive
# drive.mount('/content/drive')
# dataset = zipfile.ZipFile("/content/drive/My Drive/Dissertation/Imgs/Slim/dataset_slim.zip", 'r')
# dataset.extractall("/content/Imgs/")
# dataset.close()

In [ ]:
# Colab runtime environment #
# BASE_DIR = "../content/drive/My Drive/Dissertation"
# _data = fp.read_csv("../content/drive/My Drive/Dissertation/data_slim.csv")

In [ ]:
# Local runtime environment #
BASE_DIR = "."
_data = fp.read_csv("./data_slim.csv") # local

In [ ]:
# # Disinclude classes
# removableFindings = ["Hernia", "Pneumonia", "Edema", "Fibrosis", "Emphysema", "|", "Cardiomegaly", "Pleural_Thickening", "Consolidation", "Mass", "Pneumothorax", "Nodule", "Effusion", "Atelectasis"]
# # Collect
# removable = []
# for index, row in _data.iterrows():
#     for f in removableFindings:
#         if f in row["Finding Labels"]:
#             removable.append(index) 
# # Drop
# _data = _data.drop(removable)
# print("Removed : " + str(len(removable)))
# # Update classes
# _classes = np.unique([x.split('|')[0] for x in _data["Finding Labels"]]).tolist() # Get all UNIQUE classes

In [ ]:
# # PRE-PROCESS DATASET #
# # Only to be done once, then data_slim.csv to be used as input

# _data = fp.read_csv("./data.csv") # File containing all datapoints
# _classes = np.unique([x.split('|')[0] for x in _data["Finding Labels"]]).tolist() # Get all UNIQUE classes


# # Remove unwanted columns
# _data = _data.drop(columns=["Follow-up #", "Patient ID", "Patient Age", "Patient Gender", 
#                           "View Position", "OriginalImageWidth", "OriginalImageHeight", 
#                           "OriginalImagePixelSpacingX", "OriginalImagePixelSpacingY"])


# # Disinclude classes with low instance count
# # removableFindings = ["Hernia"]
# # # Collect
# # removable = []
# # for index, row in _data.iterrows():
# #     for f in removableFindings:
# #         if f in row["Finding Labels"]:
# #             removable.append(index) 
# # # Drop
# # _data = _data.drop(removable)
# # print("Removed : " + str(len(removable)))
# # # Update classes
# # _classes = np.unique([x.split('|')[0] for x in _data["Finding Labels"]]).tolist() # Get all UNIQUE classes


# # Reduce instance count of No Finding
# COUNT_MAX = 10000
# # Collect all instances
# removable = []
# for index, row in _data.iterrows():
#     if "No Finding" in row["Finding Labels"]:
#         removable.append(index) 

#     # Pick COUNT_MAX random instances that stay in the dataset
# for i in range(COUNT_MAX):
#     removable.remove(random.choice(removable))
# # Drop
# _data = _data.drop(removable)
# print("Removed : " + str(len(removable)))


# # Save new data file
# _data.to_csv("./data_slim.csv", index=False, mode="w")


# # Resize images
# for file in _data["Image Index"]:
#     src = "./Imgs/All/" + file
#     dest = "./Imgs/Slim_Cropped/"
#     # Resize
#     img = Image.open(src)
#     area = (100, 100, 924, 924)
#     cropped_img = img.crop(area)
# #     cropped_img.show()
#     cropped_img = cropped_img.resize((224, 224), Image.LANCZOS)
#     cropped_img.save(dest + file)

In [ ]:
# GLOBALS #
_classes = np.unique([x.split('|')[0] for x in _data["Finding Labels"]]).tolist() # Get all UNIQUE classes
session = None

# FUNCTIONS #
def SeparateClasses(set):
    # Create Classes colums - list of classes
    set["Classes"] = set["Finding Labels"].apply(lambda x:x.split("|"))
    return set


def GetUniqueClassCounts(set):
  # Find UNIQUE classes + class ID + counts
    labels = set.value_counts().to_dict()
    remove = []
    for label in labels:
        if '|' in label:
            classes = label.split('|')
            for c in classes:
                labels[c] += labels[label]
            remove.append(label)
        
    for r in remove:
        labels.pop(r)
    return labels


def GetClassWeights(set):
  # Calculate class weights
    unique_class_instances = GetUniqueClassCounts(set["Finding Labels"])
    CLASS_WEIGHTS = {}
    maxWeight = 0
    # Find max - which is going to be the base weight (1)
    for c in unique_class_instances:
        if unique_class_instances[c] > maxWeight:
            maxWeight = unique_class_instances[c]
    # Calculate individual weights
    for c in unique_class_instances:
        weight = round(1 / (unique_class_instances[c] / maxWeight), 2)
        CLASS_WEIGHTS[_classes.index(c)] = weight

    # for k, v in weights.items():
    #   print(_classes[k], ":", unique_class_instances[_classes[k]], ":", k, ":", v)
    return CLASS_WEIGHTS

# Save figures
def SaveFigure(fig, name):
    if os.path.isfile(name):
        os.remove(name)
    fig.savefig(BASE_DIR + "/Sessions/" + session + "/m_" + session + "_" + name)


# Print class counts
for c in GetUniqueClassCounts(_data["Finding Labels"]):
    print(c + " : " + str(_classes.index(c)) + " : "+ str(GetUniqueClassCounts(_data["Finding Labels"])[c]))

# Create Classes column
_data = SeparateClasses(_data)
_data.sample(5)

In [ ]:
# CREATE NEW SESSION #

session = "-"
newS = True

if not os.path.exists(BASE_DIR + "/Sessions"): # Create folder if it doesn't exist
    os.makedirs(BASE_DIR + "/Sessions")
if not os.path.exists(BASE_DIR + "/Sessions/" + session): # Create folder if it doesn't exist
    os.makedirs(BASE_DIR + "/Sessions/" + session)

In [ ]:
# LOAD A PREVIOUS SESSION #

session = "_Mobile_transfer" # Specify which session to load
newS = False

# Load session's dataset
train_set = fp.read_csv(BASE_DIR + "/Sessions/" + session + "/m_" + session + "_train.csv")
valid_set = fp.read_csv(BASE_DIR + "/Sessions/" + session + "/m_" + session + "_valid.csv")
test_set = fp.read_csv(BASE_DIR + "/Sessions/" + session + "/m_" + session + "_test.csv")
train_set = SeparateClasses(train_set)
valid_set = SeparateClasses(valid_set)
test_set = SeparateClasses(test_set)

In [ ]:
# IMAGE GENERATOR + BATCH GENERATORS #

if newS: # Split fresh dataset
    print("New Session: " + session)
    train_set, test_set = train_test_split(_data, test_size=0.2)
    train_set, valid_set = train_test_split(train_set, test_size=0.13)
else: print("Session: " + session)

print("Training size: " + str(train_set.shape[0]))
print("Valid size: " + str(valid_set.shape[0]))
print("Test size: " + str(test_set.shape[0]))

# Define image generator
img_generator = ImageDataGenerator( 
    rescale=1./255.,
    rotation_range=10,
    zoom_range=[0.85, 1.0],
    horizontal_flip=True,
)

# Properties
BATCH_SIZE = 32
IMG_COLOR = 'rgb'
IMG_SIZE = (224, 224) #(299, 299)
DIR = 'Imgs/Slim_Cropped' # Resized images dir

# Define batch generators
train_batch = img_generator.flow_from_dataframe(
        dataframe=train_set,
        directory=DIR,
        x_col="Image Index",
        y_col="Classes",
        target_size=IMG_SIZE,
        color_mode=IMG_COLOR,
        shuffle=True,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        classes=_classes)
valid_batch = img_generator.flow_from_dataframe(
        dataframe=valid_set,
        directory=DIR,
        x_col="Image Index",
        y_col="Classes",
        target_size=IMG_SIZE,
        color_mode=IMG_COLOR,
        shuffle=True,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        classes=_classes)
test_batch = ImageDataGenerator(rescale=1./255.).flow_from_dataframe(
        dataframe=test_set,
        directory=DIR,
        x_col="Image Index",
        y_col="Classes",
        target_size=IMG_SIZE,
        color_mode=IMG_COLOR,
        shuffle=False,
        batch_size=128,
        class_mode='categorical',
        classes=_classes)

# Evaluate split efficiency - every set should include all classes
train_y_unique_count = len(np.unique([x.split('|')[0] for x in train_set["Finding Labels"]]).tolist())
valid_y_unique_count = len(np.unique([x.split('|')[0] for x in valid_set["Finding Labels"]]).tolist())
test_y_unique_count = len(np.unique([x.split('|')[0] for x in test_set["Finding Labels"]]).tolist())
if train_y_unique_count != valid_y_unique_count or train_y_unique_count != test_y_unique_count:
    print("UNBALANCED SPLIT!")
else: print("BALANCED SPLIT!")

# Visualise some of the data and format
x = next(train_batch)
print("Label encoding format: ", x[1][0])
train_batch.reset()
f, img = plt.subplots(2,4, figsize = (15, 15))
plt.subplots_adjust(top=0.5, bottom=0.1, hspace=0.2, wspace=0.2)
for i in range(2):
    for j in range(4):
        if IMG_COLOR is "grayscale":
            img[i, j].imshow(np.repeat(x[0][i * 4 + j], 3, 2))

        else:
            img[i, j].imshow(x[0][i * 4 + j])
f.show()

In [ ]:
# IMPORT SESSION MODEL # 

model = load_model(BASE_DIR + "/Sessions/" + session + "/m_" + session + "_model.h5")
model.summary()
history = tf.keras.callbacks.History()
prev_history = pickle.load(open(BASE_DIR + "/Sessions/" + session + "/m_" + session + ".history", "rb")) # Load session history
EPOCHS = len(prev_history["val_accuracy"]) # Get no. epochs

# Show session AUROC if exists
if os.path.exists(BASE_DIR + "/Sessions/" + session + "/m_" + session + "_AUROC.png"):
    auroc=mpimg.imread(BASE_DIR + "/Sessions/" + session + "/m_" + session + "_AUROC.png")
    plt.figure(figsize = (10, 10))
    plt.imshow(auroc)
    plt.show()

print(EPOCHS)
# model.fit(train_batch,
#         steps_per_epoch=math.ceil(train_set.shape[0] / BATCH_SIZE), # (num_samples / batch_size)
#         epochs=30,
#         validation_data=valid_batch,
#         validation_steps=math.ceil(valid_set.shape[0] / BATCH_SIZE),
#         class_weight=GetClassWeights(train_set),
#         callbacks=[history],
#         initial_epoch=EPOCHS,
#         verbose=1)

In [ ]:
# MOBILENET #

# Load model from library
mbNet = MobileNet(input_shape=(224, 224, 3), include_top=False, weights="imagenet")
# Remove last few layers
x = GlobalAveragePooling2D()(mbNet.layers[-14].output)
# Add final activation layer
x = Dense(len(_classes), activation='sigmoid')(x)
# Compile model
model = Model(inputs=mbNet.input, outputs=x)
model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy', 'mae'])
model.summary()

earlyStop = EarlyStopping(monitor='val_mae', mode='max', min_delta=0.01, patience=5, verbose=1)
history = tf.keras.callbacks.History()

model.fit(train_batch,
        steps_per_epoch=math.ceil(train_set.shape[0] / BATCH_SIZE), #(num_samples / batch_size)
        epochs=20,
        validation_data=valid_batch,
        validation_steps=math.ceil(valid_set.shape[0] / BATCH_SIZE),
        class_weight=GetClassWeights(train_set),
        callbacks=[history],
        initial_epoch=0,
        verbose=1)

In [ ]:
# VGG #
# Load model from library
vgg = keras.applications.vgg16.VGG16(include_top=False, weights="imagenet",
                                input_tensor=None, input_shape=(224, 224, 3), pooling="avg")

# Add final activation layer
x = Dense(len(_classes), activation='softmax')(vgg.output)
# Compile model
model = Model(inputs=vgg.input, outputs=x)
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy', 'mae'])
model.summary()

earlyStop = callbacks.EarlyStopping(monitor='loss', mode='max', min_delta=0.01, patience=5, verbose=1)
history = tf.keras.callbacks.History()

model.fit(train_batch,
        steps_per_epoch=math.ceil(train_set.shape[0] / BATCH_SIZE), #(num_samples / batch_size)
        epochs=30,
        validation_data=valid_batch,
        validation_steps=math.ceil(valid_set.shape[0] / BATCH_SIZE),
        class_weight=GetClassWeights(train_set),
        callbacks=[history],
        initial_epoch=0,
        verbose=1)

In [ ]:
# RESNET #
# Load model from library
resnet = keras.applications.resnet.ResNet50(include_top=False, weights="imagenet",
                    input_tensor=None, input_shape=(224, 224, 3), pooling="avg")

# Add final activation layer
x = Dense(len(_classes), activation='sigmoid')(resnet.output)
# Compile model
model = Model(inputs= resnet.input, outputs=x)
model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy', 'mae'])
model.summary()

earlyStop = callbacks.EarlyStopping(monitor='loss', mode='max', min_delta=0.01, patience=5, verbose=1)
history = tf.keras.callbacks.History()

model.fit(train_batch,
        steps_per_epoch=math.ceil(train_set.shape[0] / BATCH_SIZE), #(num_samples / batch_size)
        epochs=30,
        validation_data=valid_batch,
        validation_steps=math.ceil(valid_set.shape[0] / BATCH_SIZE),
        class_weight=GetClassWeights(train_set),
        callbacks=[history],
        initial_epoch=0,
        verbose=1)

In [ ]:
# Display Conv layer outputs #

test_batch.reset()
x_test, y_test = next(test_batch) # Get a batch of test data
conv_layers = [x for x in model.layers if type(x) == type(Conv2D(2, (1,1)))] # Put all conv layers in a list
num_conv_layers = len([x for x in model.layers if type(x) == type(Conv2D(2, (1,1)))]) # count

f, img = plt.subplots(num_conv_layers, 5, figsize = (15, 30)) # Define figure

for imageCount in range(10): # amount of input images - how many activation maps to generate
    for j in range(num_conv_layers): # for every convolutional layer
        # Take the layers from the model
        initial_input_layer = model.input
        last_conv_layer = conv_layers[j].output

        cam_model = Model(inputs=initial_input_layer, outputs=(last_conv_layer))
        features = cam_model.predict(x_test) # Predict batch using the cam model

        image_features = features[imageCount,:,:,:] # take input first image

        plt.subplots_adjust(top=1.0, bottom=0.0, hspace=0.1, wspace=0.1)

        for i in range(5):
            featureIMG = image_features[:,:,i]
            img[j, i].imshow(featureIMG)
            img[j, i].set_yticks([])
            img[j, i].set_xticks([])
#     f.show()
    SaveFigure(f, "activation_maps#" + str(imageCount))

In [ ]:
# CLASS ACTIVATION MAP #

# model.summary()

# Get model input layer
initial_input_layer = model.input
# Get the last convolutional layer
def get_LastConvLayer():
    return [x for x in model.layers if type(x) == type(Conv2D(2, (1,1)))][-1]
last_conv_layer = get_LastConvLayer().output
# Get the classification layer
classification_layer = model.layers[-1].output
classification_layer_weights = model.layers[-1].get_weights()[0]

# Class activation map model
cam_model = Model(inputs=initial_input_layer, outputs=(last_conv_layer, classification_layer))

next(test_batch)
x_test, y_test = next(test_batch) # Get a batch of test data
features, results = cam_model.predict(x_test) # Predict batch using the cam model
print("Last conv layer feature shape: ", features.shape)

for i in range(100):
    image_features = features[i,:,:,:] # Image features recognised by the convolutional layer
    
    height_multiplier = x_test.shape[1]/image_features.shape[0] # full size / feature size
    width_multiplier = x_test.shape[2]/image_features.shape[1]

    # Apply bilinear upscaling to match the size of the input 
    cam_features = sp.ndimage.zoom(image_features, (height_multiplier, width_multiplier, 1), order=2)

    pred = np.argmax(results[i]) # Get a prediction for i
    cam_weights = classification_layer_weights[:, pred] # Get the weights from the classification layer for i
    cam_output = np.dot(cam_features, cam_weights) # Compute the dot product of the feature maps and the weights
    
    # Show / save images
#     label = "Predicted = " + _classes[pred] + ", % = " + str(results[i][pred])
    label = _classes[pred]
    fig_cam, img = plt.subplots(figsize = (6, 6))
#     fig_cam = plt.figure(figsize = (6, 6))
    plt.xlabel(label)
    img.set_yticks([])
    img.set_xticks([])
    plt.imshow(x_test[i],  alpha=0.9, extent=(0, 224, 0, 224))
    plt.imshow(cam_output, cmap="jet", alpha=0.25, extent=(0, 224, 0, 224))
    plt.show()
    plt.draw()
    SaveFigure(fig_cam, "cam#" + str(i))

In [ ]:
# EVALUATION #

In [ ]:
# Evaluate on test set
test_batch.reset()
print("Test evaluation scores: ", model.evaluate(test_batch, verbose=1))


# Get ground truth labels
mlb = MultiLabelBinarizer()
y_test = mlb.fit_transform(test_set["Classes"])


# Get predictions
test_batch.reset()
y_pred = model.predict(test_batch,
        steps=math.ceil(test_set.shape[0] / 128),
        verbose=1)

In [ ]:
# Calculate AUROC

roc_auc = roc_auc_score(y_test, y_pred)
print("AUROC score: ", roc_auc)

# Compute ROC curve and ROC area for each class
fig_auroc, fig = plt.subplots(1,1, figsize = (10, 10))
plt.plot([0, 1], [0, 1], color="black", lw=2, linestyle="dashed")
for i in range(len(_classes)):
    fpr, tpr, _ = roc_curve(y_test[:,i], y_pred[:,i]) # [:,i] column value
    roc_auc = auc(fpr, tpr)
    fig.plot(fpr, tpr, label = _classes[i] + "-AUC: " + str(round(roc_auc, 3)))
fig.legend()
fig.set_xlabel("False Positive Rate")
fig.set_ylabel("True Positive Rate")
fig.set_title("ROC Curve per Class")

In [ ]:
# Append to previous history dictionary, IF one exists
hist = history.history
if session is not None and os.path.exists(BASE_DIR + "/Sessions/" + session + "/m_" + session + ".history"):
    prev_history = pickle.load(open(BASE_DIR + "/Sessions/" + session + "/m_" + session + ".history", "rb"))
    ep = len(hist["val_accuracy"]) # Get no. epochs
    for i in range(ep): # for every epoch
        for metric in prev_history.keys(): # for every metric
            prev_history[metric].append(hist[metric][i]) # append the epoch's metrics to the prev. hist.
    hist = prev_history

# Plot accuracy
fig_accuracy = plt.figure(figsize = (8, 5))
plt.plot(hist["accuracy"])
plt.plot(hist['val_accuracy'])
plt.title("Training Accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Accuracy","Validation Accuracy"])
plt.show()
# Plot loss
fig_loss = plt.figure(figsize = (8, 5))
plt.plot(hist['loss'])
plt.plot(hist['val_loss'])
plt.title("Training Loss ")
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.legend(["Loss","Validation Loss"])
plt.show()
# Plot MAE
fig_mae = plt.figure(figsize = (8, 5))
plt.plot(hist['mae'])
plt.plot(hist['val_mae'])
plt.title("Training MAE")
plt.ylabel("MAE")
plt.xlabel("Epoch")
plt.legend(["MAE","Validation MAE"])
plt.show()
plt.draw()

In [ ]:
# SAVE SESSION #

# Save confusion matrix to file
conf_matrix = confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))
with open(BASE_DIR + "/Sessions/" + session + "/m_" + session + "_confusionM.txt", "w") as f:
    f.write(np.array2string(conf_matrix))
    f.close()

# Save dataset
train_set = train_set.drop(columns=["Classes"]) # Drop bad columns - comma separation not compatible with import split
valid_set = valid_set.drop(columns=["Classes"])
test_set = test_set.drop(columns=["Classes"])
train_set.to_csv(BASE_DIR + "/Sessions/" + session + "/m_" + session + "_train.csv", index=False, mode="w")
valid_set.to_csv(BASE_DIR + "/Sessions/" + session + "/m_" + session + "_valid.csv", index=False, mode="w")
test_set.to_csv(BASE_DIR + "/Sessions/" + session + "/m_" + session + "_test.csv", index=False, mode="w")

# Append to previous history dictionary, IF one exists
# NOTE: file exist check will fail if the session is being renamed, comment out next line and let hist variable come from the above cell
hist = history.history
if os.path.exists(BASE_DIR + "/Sessions/" + session + "/m_" + session + ".history"):
    prev_history = pickle.load(open(BASE_DIR + "/Sessions/" + session + "/m_" + session + ".history", "rb"))
    ep = len(hist["val_accuracy"]) # Get no. epochs
    for i in range(ep): # for every epoch
        for metric in prev_history.keys(): # for every metric
            prev_history[metric].append(hist[metric][i]) # append the epoch's metrics to the prev. hist.
    hist = prev_history
# Save training history obj. as binary file
with open(BASE_DIR + "/Sessions/" + session + "/m_" + session + ".history", "wb") as f:
    pickle.dump(hist, f)
    f.close()

# Save model - includes structure, weights, optimizer weights..
model.save(BASE_DIR + "/Sessions/" + session + "/m_" + session + "_model.h5", overwrite=True) # Model

SaveFigure(fig_auroc, "AUROC")
SaveFigure(fig_accuracy, "accuracy_history")
SaveFigure(fig_loss, "loss_history")
SaveFigure(fig_mae, "mae_history")